In [24]:
import os
import tempfile

import keras
import numpy as np
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import Dense, Activation, Flatten
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.utils.np_utils import to_categorical

import pandas as pd
import matplotlib.pyplot as plt

import gensim
from gensim.models import Word2Vec

In [3]:
df = pd.read_csv('train.csv')

In [4]:
X_train, y_train = df['text'], df['author']

In [5]:
len(X_train), len(y_train)

(19579, 19579)

## Embedding: generate vectors

In [6]:
def process(string_list):
    new_list = [s.replace('.', '').replace(',', '') for s in string_list]
    new_list = [s.lower().split() for s in new_list]
    return new_list

In [7]:
sentences = process([x for x in X_train])
model = Word2Vec(sentences, size=100, window=10, min_count=1, workers=4)
word_vectors = model.wv
del model

In [8]:
def vectorize_word(word):
    try:
        return word_vectors[word.strip(',.!?\'"').strip(',.!?\'"').lower()]
    except:
        return word_vectors['the']
                            

def vectorize(text):
    return [vectorize_word(w) for w in text.split()]

X_train_vec = [vectorize(text) for text in X_train]

In [9]:
df

,id,text,author
0,id26305,"This process, however, afforded me no means of...",EAP
1,id17569,It never once occurred to me that the fumbling...,HPL
2,id11008,"In his left hand was a gold snuff box, from wh...",EAP
3,id27763,How lovely is spring As we looked from Windsor...,MWS
4,id12958,"Finding nothing else, not even gold, the Super...",HPL
5,id22965,"A youth passed in solitude, my best years spen...",MWS
6,id09674,"The astronomer, perhaps, at this point, took r...",EAP
7,id13515,The surcingle hung in ribands from my body.,EAP
8,id19322,I knew that you could not say to yourself 'ste...,EAP
9,id00912,I confess that neither the structure of langua...,MWS


In [10]:
print len(y_train)
y_train = [{'EAP': 0, 'HPL': 1, 'MWS': 2}[y] for y in y_train]
print len(y_train)

19579
19579


In [11]:
X_train_vec = np.array(X_train_vec)
y_train = to_categorical(np.array(y_train), num_classes=3)

In [12]:
len(y_train)

19579

In [13]:
# truncate and pad input sequences
X_train = X_train_vec
max_length = 500
X_train = sequence.pad_sequences(X_train, maxlen=max_length)

In [14]:
X_train.shape

(19579, 500, 100)

In [25]:
model = Sequential()
model.add(Dense(512, input_shape=(500, 100), init="uniform",
	activation="relu"))
model.add(Dense(256, init="uniform", activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(256, init="uniform", activation="relu"))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(3, activation="softmax"))
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=10, validation_split=0.33, batch_size=32)

/Users/emannuelcarvalho/.virtualenvs/ml/lib/python2.7/site-packages/ipykernel_launcher.py:3: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(512, activation="relu", kernel_initializer="uniform", input_shape=(500, 100))`
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/emannuelcarvalho/.virtualenvs/ml/lib/python2.7/site-packages/ipykernel_launcher.py:4: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(256, activation="relu", kernel_initializer="uniform")`
  after removing the cwd from sys.path.
/Users/emannuelcarvalho/.virtualenvs/ml/lib/python2.7/site-packages/ipykernel_launcher.py:6: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(256, activation="relu", kernel_initializer="uniform")`
  


Train on 13117 samples, validate on 6462 samples
Epoch 1/10
13117/13117 [==============================] - 353s - loss: 0.9977 - acc: 0.5087 - val_loss: 0.9809 - val_acc: 0.5351
Epoch 2/10
13117/13117 [==============================] - 351s - loss: 0.8975 - acc: 0.5853 - val_loss: 0.9475 - val_acc: 0.5661
Epoch 3/10
13117/13117 [==============================] - 352s - loss: 0.8356 - acc: 0.6277 - val_loss: 0.8429 - val_acc: 0.6261
Epoch 4/10
13117/13117 [==============================] - 351s - loss: 0.7953 - acc: 0.6467 - val_loss: 0.8590 - val_acc: 0.6105
Epoch 5/10
13117/13117 [==============================] - 354s - loss: 0.7650 - acc: 0.6654 - val_loss: 0.8526 - val_acc: 0.6227
Epoch 6/10
13117/13117 [==============================] - 352s - loss: 0.7431 - acc: 0.6716 - val_loss: 0.8274 - val_acc: 0.6329
Epoch 7/10
13117/13117 [==============================] - 352s - loss: 0.7220 - acc: 0.6875 - val_loss: 0.8572 - val_acc: 0.6413
Epoch 8/10
13117/13117 [========================

In [26]:
prediction = model.predict(np.array([X_train[3]]))
print prediction, y_train[3]

[[ 0.48820114  0.15095983  0.36083898]] [ 0.  0.  1.]


In [27]:
test_df = pd.read_csv('test.csv')
X_test_text = test_df['text']

In [28]:
X_test = [vectorize(text) for text in X_test_text]
X_test = sequence.pad_sequences(X_test, maxlen=max_length)

In [29]:
ids, y_test = [i for i in test_df['id']], model.predict(X_test)

In [30]:
# submission_file = open('submission1.csv', 'w')
file_text = ""
file_text += "id,EAP,HPL,MWS\n"
for data in zip(ids, y_test):
    file_text += "{},{},{},{}\n".format(data[0], data[1][0], data[1][1], data[1][2])
print(file_text[:200])

id,EAP,HPL,MWS
id02310,0.0450474508107,0.0615367554128,0.893415808678
id24541,0.969961106777,0.0161529406905,0.0138858770952
id00134,0.0378526486456,0.716230392456,0.245916977525
id27757,0.83968108892


In [21]:
submission_file = open('submission3.csv', 'w')
submission_file.write(file_text)
submission_file.close()